In [33]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path

# system paths
import sys
home = str(Path.home())
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/soniame/.conda/envs/norm_env'
from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

# aws connection
import boto3
import awswrangler as wr

# local libraries
from src import utils

# analysis libraries
import pandas as pd
import osmpy
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection, shape
from shapely import wkt
from shapely.ops import transform
import shapely

In [2]:
conn = utils.connect_athena(path='../configs/athena.yaml')

../src/utils.py:159: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  athena_config = yaml.load(open(path, "r"))


In [3]:
df_sample = pd.read_sql_query(f"""
    select *
    from spd_sdv_waze_corona.raw_copy_jams
    limit 100""", conn)

In [41]:
#polygon = 'POLYGON((2.0117187499999822 44.38657313925715,-19.433593750000018 19.207272119703983,19.414062499999982 6.904449621538131,64.94140624999999 -3.096801256840523,81.46484374999999 37.21269961002643,45.78124999999998 24.106495997107682,53.69140624999998 51.22054369437158,3.7695312499999822 37.07257833232809,2.0117187499999822 44.38657313925715))'
#geometry = wkt.loads(polygon)
#geometry

In [88]:
polygon = Polygon([(0, 0), (5, 5), (5, 0)])
print(polygon.area)

12.5


In [ ]:
geometry = polygon
result = osmpy.core.katana(geometry, osmpy.core.threshold_func, 50, 100)
multipolygon = MultiPolygon(result)

In [ ]:
print(len(multipolygon.geoms))
multipolygon

In [87]:
grid = list()
for polygon in multipolygon:  # same for multipolygon.geoms
    grid.append(str(polygon))
grid    

['POLYGON ((0 0, 0.3125 0.3125, 0.3125 0, 0 0))',
 'POLYGON ((0.625 0.15625, 0.625 0, 0.3125 0, 0.3125 0.15625, 0.625 0.15625))',
 'POLYGON ((0.3125 0.3125, 0.625 0.3125, 0.625 0.15625, 0.3125 0.15625, 0.3125 0.3125))',
 'POLYGON ((0.3125 0.3125, 0.625 0.625, 0.625 0.3125, 0.3125 0.3125))',
 'POLYGON ((0.9375 0, 0.625 0, 0.625 0.15625, 0.9375 0.15625, 0.9375 0))',
 'POLYGON ((0.625 0.15625, 0.625 0.3125, 0.9375 0.3125, 0.9375 0.15625, 0.625 0.15625))',
 'POLYGON ((1.25 0.15625, 1.25 0, 0.9375 0, 0.9375 0.15625, 1.25 0.15625))',
 'POLYGON ((1.25 0.3125, 1.25 0.15625, 0.9375 0.15625, 0.9375 0.3125, 1.25 0.3125))',
 'POLYGON ((0.9375 0.46875, 0.9375 0.3125, 0.625 0.3125, 0.625 0.46875, 0.9375 0.46875))',
 'POLYGON ((0.625 0.625, 0.9375 0.625, 0.9375 0.46875, 0.625 0.46875, 0.625 0.625))',
 'POLYGON ((1.25 0.46875, 1.25 0.3125, 0.9375 0.3125, 0.9375 0.46875, 1.25 0.46875))',
 'POLYGON ((0.9375 0.625, 1.25 0.625, 1.25 0.46875, 0.9375 0.46875, 0.9375 0.625))',
 'POLYGON ((0.625 0.625, 0.9375